<a href="https://colab.research.google.com/github/PurnamaRidzkyN/Titanic_Prediction/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240703%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240703T033959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38eeb6f2f4040f62a4c851ff69d9853567e0a11102321211f95a24e9dd5a5e8826dfa7ef40d0f1072b59981ac424cb68ffa866f6cb80a891e389e79792854d29a21e3982decb1441d3ca0041e93416455d656b419ac609350f04b03cc9d7a47cc6c0a8bfd88baa67dd48a3003f2a898bb2f264573b1ddff90e8705371dfb10bc111825343d4b9492497fb7553add54034c96da1876d7836755b345a77682599bb8eaf96cb732433f32de9561f897461a9b5290db3680417d3fd55bb7879a15d7efa72f65ce9d19bb3d4f8d15c15039f224f7a6fff4111adecc0bdc56f60ee3f9c684e8b991552ff4900668f0e7b724f05ceae6c70fd12b65f0e5a1a331ad308e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

data = '/kaggle/input/titanic/train.csv'
df = pd.read_csv(data)

In [ ]:
df.info()

In [ ]:
from sklearn.pipeline import Pipeline
numeric_columns = df.select_dtypes(include=['float64','int64'])

correlation_matrix = numeric_columns.corr()

plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix, cmap="YlGnBu", annot=True, linewidths=1.0)
plt.title("Correlation between Numeric columns in the Dataset")
plt.show()

In [ ]:
df.info()
start_train_set = df

In [ ]:
plt.subplot(1,2,1)
start_train_set['Survived'].hist()
start_train_set['Pclass'].hist()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class ageImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        imputer = SimpleImputer(strategy="mean")
        X_copy = X.copy()
        X_copy['Age'] = imputer.fit_transform(X['Age'].values.reshape(-1, 1))
        return X_copy

In [ ]:
from sklearn.preprocessing import OneHotEncoder

class FeatureEncoder(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self,X):
        encoder = OneHotEncoder()
        matrix = encoder.fit_transform(X[['Embarked']]).toarray()

        column_names = ["C","S","Q","N"]

        for i in range(len(matrix.T)):
            X[column_names[i]]= matrix.T[i]

        matrix = encoder.fit_transform(X[['Sex']]).toarray()

        column_names = ["Female", "Male"]

        for i in range (len(matrix.T)):
            X[column_names[i]] = matrix.T[i]

        return X

In [ ]:
class FeatureDropper(BaseEstimator, TransformerMixin):
    def fit(self,X,y= None):
        return self
    def transform(self,X):
        return X.drop(["Embarked","Name","Ticket","Cabin","Sex","SibSp"],axis= 1,errors="ignore")

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([("ageimputer", ageImputer()),
                      ("featureencoder", FeatureEncoder()),
                      ("featuredropper", FeatureDropper())])

In [ ]:
sns.boxplot(x="Fare", data=start_train_set)

In [ ]:
sns.boxplot(x="Age", data= start_train_set)

In [ ]:
from sklearn.preprocessing import StandardScaler
class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, attributes):
        self.attributes = attributes
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X[self.attributes])
        return self

    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.attributes] = self.scaler.transform(X[self.attributes])
        return X_transformed

In [ ]:
attributes_to_normalize = ['Age', 'Fare']

# Membuat pipeline dengan normalisasi Z-score dan StandardScaler
pipeline = Pipeline([("ageimputer", ageImputer()),
                      ("featureencoder", FeatureEncoder()),
                      ("featuredropper", FeatureDropper()),
                    ('scaler', CustomScaler(attributes=attributes_to_normalize))])
# Melakukan normalisasi menggunakan pipeline

final_train = pipeline.fit_transform(start_train_set)

# Hasil normalisasi
final_train.head()

In [ ]:
final_train.info()

In [ ]:
numeric_columns = final_train.select_dtypes(include=['float64','int64'])

correlation_matrix = numeric_columns.corr()

plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix, cmap="YlGnBu", annot=True, linewidths=1.0)
plt.title("Correlation between Numeric columns in the Dataset")
plt.show()

In [ ]:
final_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Contoh data yang sudah diproses
X = final_train.drop(['Survived','N'], axis=1)
y = final_train['Survived']

# Pisahkan dataset menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Inisialisasi dan latih model RandomForestClassifier
rf_model = RandomForestClassifier(criterion ="gini",n_estimators=250)
rf_model.fit(X_train, y_train)

# Lakukan prediksi pada data pengujian
y_pred_rf = rf_model.predict(X_test)

# Hitung dan cetak akurasi
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("RandomForestClassifier Accuracy Score:", accuracy_rf)

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
final_test_data = pipeline.fit_transform(test_data)

In [ ]:
final_test_data=final_test_data.fillna(0)

In [ ]:
final_test_data.info()

In [ ]:
predict = rf_model.predict(final_test_data)

In [ ]:
# Buat DataFrame baru untuk hasil prediksi
predictions_df = pd.DataFrame({
    'PassengerId': final_test_data['PassengerId'],  # Ambil PassengerId dari data uji
    'Survived': predict  # Masukkan hasil prediksi
})

# Simpan DataFrame ke dalam file CSV
predictions_df.to_csv('/kaggle/working/predictions.csv', index=False)